In [1]:
SEED = 61


import os
import re
import gc
import h5py
import torch
import numpy as np
import pandas as pd
import tensorflow as tf
import random as python_random
import matplotlib.pyplot as plt
import tensorflow_addons as tfa

from tqdm import tqdm
from nltk import tokenize

from sklearn import preprocessing
from sklearn.decomposition import PCA

from keras import backend as K
from keras import initializers,regularizers,constraints
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Reshape, Input, Embedding, Flatten, Dense, Dropout, BatchNormalization, Activation #, merge
from keras.layers import TimeDistributed, LSTM, GRU, Bidirectional, Convolution1D, MaxPooling1D, MaxPooling2D
from keras.layers.core import RepeatVector, Reshape
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import Sequential, Model, load_model
from sklearn.model_selection import cross_val_score

def reset_seeds():
    np.random.seed(SEED) 
    python_random.seed(SEED)
    tf.random.set_seed(SEED)
    os.environ["PYTHONHASHSEED"] = str(SEED)

# from tensorflow.python.keras.layers import Layer, InputSpec, Lambda
# from tensorflow.keras import Model
# from attention import Attention_input1, Attention_input2
# from keras.optimizers import SGD, RMSprop, Adagrad

In [2]:
def read_hdf5(path):
    read_file = h5py.File(path, 'r')
    
    feature_names = list(read_file.keys())
    loaded_data = []
    
    for name in feature_names:
        dataset = read_file[name][:]
        if dataset.dtype == np.dtype('object'):
            dataset = np.array([x.decode('UTF-8') for x in dataset])            
        loaded_data.append((name, dataset))

    return loaded_data

def loadz(path):
    data = np.load(path)['arr_0']
    return data

In [3]:
def merge_mvsa(mvsa_single, mvsa_multiple):
    mvsa = np.concatenate((mvsa_single, mvsa_multiple), axis=0)
    return mvsa

def load_mvsa_feature(feature_name, merge=False):
    folder_path = os.path.join('../input/mvsa-features/', feature_name)
    single_file = 'mvsa-single-{}.npz'.format(feature_name)
    multiple_file = 'mvsa-multiple-{}.npz'.format(feature_name)
    mvsa_single = loadz(os.path.join(folder_path, single_file))
    mvsa_multiple = loadz(os.path.join(folder_path, multiple_file))
    
    if merge == True:
        return merge_mvsa(mvsa_single, mvsa_multiple)
    
    return mvsa_single, mvsa_multiple

def load_labels(path):
    data = read_hdf5(path)

    for x in data:
        if x[0] == 'multimodal-labels':
            labels = x[1]
        if x[0] == 'text-labels':
            text_labels = x[1]
        if x[0] == 'image-labels':
            image_labels = x[1]
        
    return labels, text_labels, image_labels

In [4]:
def plot_metrics(history):
    fig = plt.figure(figsize=(20, 5))

    fig.add_subplot(1, 3, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('LOSS')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    fig.add_subplot(1, 3, 2)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('ACCURACY')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    fig.add_subplot(1, 3, 3)
    plt.plot(history.history['f1_score'])
    plt.plot(history.history['val_f1_score'])
    plt.title('F1-SCORE')
    plt.ylabel('f1-score')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    plt.show()

In [5]:
# e.g. validation_split=0.1 -----> 8:1:1 ratio of train, val, test
def split_data(data, validation_split):
    num_val = int(validation_split * data.shape[0])
    data_train = data[:-(num_val*2)]
    data_val = data[-(num_val*2):-(num_val)]
    data_test = data[-num_val:]
    return data_train, data_val, data_test

In [6]:
def create_model_pretrained(input_shape):
    f1_score = tfa.metrics.F1Score(num_classes=NUM_CLASSES, average='micro')
    
    image_input = Input(shape=input_shape)
    outputs = Dense(NUM_CLASSES, activation='softmax') (image_input)
    
    model = Model(image_input, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', f1_score]) # f1 #tf.keras.metrics.AUC()
    return model

In [7]:
def evaluate_model(model, X_test, y_test, checkpoint=None, verbose=1):
    if checkpoint is not None:
        model = load_model('./model_checkpoint/{}.h5'.format(checkpoint))#, custom_objects={'f1': f1})
    
    loss, acc, f1 = model.evaluate(X_test, y_test, verbose=verbose)
    
    if verbose == 1:
        print('Loss:', loss)
        print('Accuracy:', acc)
        print('F1-score:', f1)
        
    return loss, acc, f1

In [8]:
def run_and_evaluate(name, X, y, verbose=0):
    y = le.fit_transform(y)
    y = to_categorical(np.asarray(y))
    
    X_train, X_val, X_test = split_data(X, VALIDATION_SPLIT)
    y_train, y_val, y_test = split_data(y, VALIDATION_SPLIT)
    
    model = create_model_pretrained(X_train.shape[1:])
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=EARLY_STOPPING)
    checkpoint = ModelCheckpoint('./model_checkpoint/{}.h5'.format(name), save_best_only=True, verbose=verbose)
    
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), 
                                   epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=verbose,
                                   callbacks=[checkpoint, early_stopping])
    print(early_stopping.stopped_epoch)
    return history, evaluate_model(model, X_test, y_test, checkpoint=name, verbose=verbose)

In [9]:
def style_dataframe(dataframe):
    return dataframe.style.highlight_max(subset=['Accuracy', 'F1-score'], props='color:lightgreen', axis=0)\
                          .highlight_min(subset=['Accuracy', 'F1-score'], props='color:tomato', axis=0)\
                          .highlight_min(subset=['Loss'], props='color:lightgreen', axis=0)\
                          .highlight_max(subset=['Loss'], props='color:tomato', axis=0)

In [10]:
from IPython.display import display_html

def display_dataframes(dfs, names=[], index=False):
    def to_df(x):
        if isinstance(x, pd.Series):
            return pd.DataFrame(x)
        else:
            return x
    html_str = ''
    if names:
        html_str += ('<tr>' + 
                     ''.join(f'<td style="text-align:center">{name}</td>' for name in names) + 
                     '</tr>')
    html_str += ('<tr>' + 
                 ''.join(f'<td style="vertical-align:top"> {to_df(df).to_html()}</td>' 
                         for df in dfs) + 
                 '</tr>')
    html_str = f'<table>{html_str}</table>'
    html_str = html_str.replace('table','table style="display:inline"')
    display_html(html_str, raw=True)

# Load data

In [11]:
# load separate
mvsa_single_vgg16, mvsa_multiple_vgg16 = load_mvsa_feature('vgg16')
mvsa_single_vgg19, mvsa_multiple_vgg19 = load_mvsa_feature('vgg19')
mvsa_single_resnet50, mvsa_multiple_resnet50 = load_mvsa_feature('resnet50')
mvsa_single_resnet101, mvsa_multiple_resnet101 = load_mvsa_feature('resnet101')
mvsa_single_resnet152, mvsa_multiple_resnet152 = load_mvsa_feature('resnet152')
mvsa_single_densenet121, mvsa_multiple_densenet121 = load_mvsa_feature('densenet121')
mvsa_single_densenet169, mvsa_multiple_densenet169 = load_mvsa_feature('densenet169')
mvsa_single_densenet201, mvsa_multiple_densenet201 = load_mvsa_feature('densenet201')

In [12]:
# load merge
mvsa_vgg16 = load_mvsa_feature('vgg16', merge=True)
mvsa_vgg19 = load_mvsa_feature('vgg19', merge=True)
mvsa_resnet50 = load_mvsa_feature('resnet50', merge=True)
mvsa_resnet101 = load_mvsa_feature('resnet101', merge=True)
mvsa_resnet152 = load_mvsa_feature('resnet152', merge=True)
mvsa_densenet121 = load_mvsa_feature('densenet121', merge=True)
mvsa_densenet169 = load_mvsa_feature('densenet169', merge=True)
mvsa_densenet201 = load_mvsa_feature('densenet201', merge=True)

In [13]:
mvsa_single_multimodal_labels, mvsa_single_text_labels, mvsa_single_image_labels = load_labels('../input/mvsa-features/labels/mvsa-single-labels.hdf5')
mvsa_multiple_multimodal_labels, mvsa_multiple_text_labels, mvsa_multiple_image_labels = load_labels('../input/mvsa-features/labels/mvsa-multiple-labels.hdf5')

mvsa_multimodal_labels = merge_mvsa(mvsa_single_multimodal_labels, mvsa_multiple_multimodal_labels)
mvsa_text_labels = merge_mvsa(mvsa_single_text_labels, mvsa_multiple_text_labels)
mvsa_image_labels = merge_mvsa(mvsa_single_image_labels, mvsa_multiple_image_labels)

In [14]:
le = preprocessing.LabelEncoder()
le.fit(mvsa_multimodal_labels)
NUM_CLASSES = len(le.classes_) # =3

In [15]:
# prepare all features data
pretrained_names = ['vgg16', 'vgg19', 'resnet50', 'resnet101', 'resnet152', 'densenet121', 'densenet169', 'densenet201']
# single_pretrained_names = ['single-' + name for name in pretrained_names]
# multiple_pretrained_names = ['multiple-' + name for name in pretrained_names]

mvsa_single_features = [mvsa_single_vgg16, mvsa_single_vgg19, 
                        mvsa_single_resnet50, mvsa_single_resnet101, mvsa_single_resnet152, 
                        mvsa_single_densenet121, mvsa_single_densenet169, mvsa_single_densenet201]

mvsa_multiple_features = [mvsa_multiple_vgg16, mvsa_multiple_vgg19, 
                          mvsa_multiple_resnet50, mvsa_multiple_resnet101, mvsa_multiple_resnet152, 
                          mvsa_multiple_densenet121, mvsa_multiple_densenet169, mvsa_multiple_densenet201]

mvsa_features = [mvsa_vgg16, mvsa_vgg19, 
                 mvsa_resnet50, mvsa_resnet101, mvsa_resnet152, 
                 mvsa_densenet121, mvsa_densenet169, mvsa_densenet201]

# Run models and Evalution display

In [16]:
reset_seeds()
EPOCHS = 100
BATCH_SIZE = 128
VALIDATION_SPLIT = 0.1
EARLY_STOPPING = 10

## With original image labels

In [17]:
print('With original image labels')
# run model with all features data of MVSA-Single
mvsa_single_hitories = []
mvsa_single_score = []
for i in range(len(pretrained_names)):
    print('MVSA-Single:', pretrained_names[i])
    history, scores = run_and_evaluate('single-' + pretrained_names[i], mvsa_single_features[i], mvsa_single_image_labels, verbose=0)
    mvsa_single_hitories.append(history)
    mvsa_single_score.append(scores)
    print()
    
df_single_scores = pd.DataFrame(mvsa_single_score, columns=['Loss', 'Accuracy', 'F1-score'], index=pretrained_names)

With original image labels
MVSA-Single: vgg16


2022-06-30 13:46:03.990428: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-30 13:46:03.991551: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-30 13:46:03.992223: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-30 13:46:03.993054: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

13

MVSA-Single: vgg19
26

MVSA-Single: resnet50
24

MVSA-Single: resnet101
59

MVSA-Single: resnet152
24

MVSA-Single: densenet121
22

MVSA-Single: densenet169
16

MVSA-Single: densenet201
16



In [18]:
print('With original image labels')
# run model with all features data of MVSA-Multiple
mvsa_multiple_hitories = []
mvsa_multiple_score = []
for i in range(len(pretrained_names)):
    print('MVSA-Multiple:', pretrained_names[i])
    history, scores = run_and_evaluate('multiple-' + pretrained_names[i], mvsa_multiple_features[i], mvsa_multiple_image_labels, verbose=0)
    mvsa_multiple_hitories.append(history)
    mvsa_multiple_score.append(scores)
    print()
    
df_multiple_scores = pd.DataFrame(mvsa_multiple_score, columns=['Loss', 'Accuracy', 'F1-score'], index=pretrained_names)

With original image labels
MVSA-Multiple: vgg16
18

MVSA-Multiple: vgg19
11

MVSA-Multiple: resnet50
17

MVSA-Multiple: resnet101
20

MVSA-Multiple: resnet152
20

MVSA-Multiple: densenet121
18

MVSA-Multiple: densenet169
16

MVSA-Multiple: densenet201
16



In [19]:
mvsa_average_scores = np.mean([mvsa_single_score, mvsa_multiple_score], axis=0)
df_average_scores = pd.DataFrame(mvsa_average_scores, columns=['Loss', 'Accuracy', 'F1-score'], index=pretrained_names)

In [20]:
print('With original image labels\n')
display_dataframes((style_dataframe(df_single_scores), style_dataframe(df_multiple_scores), style_dataframe(df_average_scores)), 
                   names=['MVSA-Single', 'MVSA-Multiple', 'Average'])

With original image labels



MVSA-Single MVSA-Multiple Average 
 
 
 
   
 Loss 
 Accuracy 
 F1-score 
 
 
 
 
 vgg16 
 0.865558 
 0.577726 
 0.577726 
 
 
 vgg19 
 0.859385 
 0.580046 
 0.580046 
 
 
 resnet50 
 0.930690 
 0.580046 
 0.580046 
 
 
 resnet101 
 0.927502 
 0.577726 
 0.577726 
 
 
 resnet152 
 0.958335 
 0.561485 
 0.561485 
 
 
 densenet121 
 0.907156 
 0.605568 
 0.605568 
 
 
 densenet169 
 0.928015 
 0.577726 
 0.577726 
 
 
 densenet201 
 0.864275 
 0.596288 
 0.596288 
 
 
 
 
 
 
 
   
 Loss 
 Accuracy 
 F1-score 
 
 
 
 
 vgg16 
 0.924527 
 0.548230 
 0.548230 
 
 
 vgg19 
 0.913936 
 0.571429 
 0.571429 
 
 
 resnet50 
 0.911810 
 0.554945 
 0.554945 
 
 
 resnet101 
 0.923054 
 0.547009 
 0.547009 
 
 
 resnet152 
 0.919318 
 0.550672 
 0.550672 
 
 
 densenet121 
 0.894839 
 0.564103 
 0.564103 
 
 
 densenet169 
 0.889975 
 0.567766 
 0.567766 
 
 
 densenet201 
 0.897645 
 0.581197 
 0.581197 
 
 
 
 
 
 
 
   
 Loss 
 Accuracy 
 F1-score 
 
 
 
 
 vgg16 
 0.895043 
 0.562978 
 0.562978 
 
 
 vgg19 
 0.886660 
 0.575738 
 0.575738 
 
 
 resnet50 
 0.921250 
 0.567496 
 0.567496 
 
 
 resnet101 
 0.925278 
 0.562367 
 0.562367 
 
 
 resnet152 
 0.938826 
 0.556078 
 0.556078 
 
 
 densenet121 
 0.900998 
 0.584836 
 0.584836 
 
 
 densenet169 
 0.908995 
 0.572746 
 0.572746 
 
 
 densenet201 
 0.880960 
 0.588742 
 0.588742

## With multimodal labels

In [21]:
print('With multimodal labels')

# run model with all features data of MVSA-Single
mvsa_single_hitories2 = []
mvsa_single_score2 = []
for i in range(len(pretrained_names)):
    print('MVSA-Single:', pretrained_names[i])
    history, scores = run_and_evaluate('single-' + pretrained_names[i], mvsa_single_features[i], mvsa_single_multimodal_labels, verbose=0)
    mvsa_single_hitories2.append(history)
    mvsa_single_score2.append(scores)
    print()
    
df_single_scores2 = pd.DataFrame(mvsa_single_score2, columns=['Loss', 'Accuracy', 'F1-score'], index=pretrained_names)

With multimodal labels
MVSA-Single: vgg16
16

MVSA-Single: vgg19
13

MVSA-Single: resnet50
22

MVSA-Single: resnet101
22

MVSA-Single: resnet152
28

MVSA-Single: densenet121
22

MVSA-Single: densenet169
12

MVSA-Single: densenet201
14



In [22]:
print('With multimodal labels')
# run model with all features data of MVSA-Multiple
mvsa_multiple_hitories2 = []
mvsa_multiple_score2 = []
for i in range(len(pretrained_names)):
    print('MVSA-Multiple:', pretrained_names[i])
    history, scores = run_and_evaluate('multiple-' + pretrained_names[i], mvsa_multiple_features[i], mvsa_multiple_multimodal_labels, verbose=0)
    mvsa_multiple_hitories2.append(history)
    mvsa_multiple_score2.append(scores)
    print()
    
df_multiple_scores2 = pd.DataFrame(mvsa_multiple_score2, columns=['Loss', 'Accuracy', 'F1-score'], index=pretrained_names)

With multimodal labels
MVSA-Multiple: vgg16
18

MVSA-Multiple: vgg19
18

MVSA-Multiple: resnet50
18

MVSA-Multiple: resnet101
20

MVSA-Multiple: resnet152
20

MVSA-Multiple: densenet121
16

MVSA-Multiple: densenet169
16

MVSA-Multiple: densenet201
13



In [23]:
mvsa_average_scores2 = np.mean([mvsa_single_score2, mvsa_multiple_score2], axis=0)
df_average_scores2 = pd.DataFrame(mvsa_average_scores2, columns=['Loss', 'Accuracy', 'F1-score'], index=pretrained_names)

In [24]:
print('With multimodal labels\n')
display_dataframes((style_dataframe(df_single_scores2), style_dataframe(df_multiple_scores2), style_dataframe(df_average_scores2)), 
                   names=['MVSA-Single', 'MVSA-Multiple', 'Average'])

With multimodal labels



MVSA-Single MVSA-Multiple Average 
 
 
 
   
 Loss 
 Accuracy 
 F1-score 
 
 
 
 
 vgg16 
 0.760666 
 0.628770 
 0.628770 
 
 
 vgg19 
 0.757656 
 0.617169 
 0.617169 
 
 
 resnet50 
 0.796381 
 0.610209 
 0.610209 
 
 
 resnet101 
 0.803058 
 0.612529 
 0.612529 
 
 
 resnet152 
 0.817103 
 0.603248 
 0.603248 
 
 
 densenet121 
 0.835543 
 0.591647 
 0.591647 
 
 
 densenet169 
 0.840911 
 0.598608 
 0.598608 
 
 
 densenet201 
 0.831024 
 0.587007 
 0.587007 
 
 
 
 
 
 
 
   
 Loss 
 Accuracy 
 F1-score 
 
 
 
 
 vgg16 
 0.891924 
 0.638584 
 0.638584 
 
 
 vgg19 
 0.901875 
 0.645910 
 0.645910 
 
 
 resnet50 
 0.903102 
 0.639805 
 0.639805 
 
 
 resnet101 
 0.905602 
 0.639805 
 0.639805 
 
 
 resnet152 
 0.905639 
 0.639805 
 0.639805 
 
 
 densenet121 
 0.891714 
 0.630647 
 0.630647 
 
 
 densenet169 
 0.877630 
 0.626374 
 0.626374 
 
 
 densenet201 
 0.866111 
 0.634310 
 0.634310 
 
 
 
 
 
 
 
   
 Loss 
 Accuracy 
 F1-score 
 
 
 
 
 vgg16 
 0.826295 
 0.633677 
 0.633677 
 
 
 vgg19 
 0.829766 
 0.631540 
 0.631540 
 
 
 resnet50 
 0.849742 
 0.625007 
 0.625007 
 
 
 resnet101 
 0.854330 
 0.626167 
 0.626167 
 
 
 resnet152 
 0.861371 
 0.621526 
 0.621526 
 
 
 densenet121 
 0.863629 
 0.611147 
 0.611147 
 
 
 densenet169 
 0.859271 
 0.612491 
 0.612491 
 
 
 densenet201 
 0.848567 
 0.610659 
 0.610659

# With merge MVSA data

In [25]:
print('With original image labels')
mvsa_hitories3 = []
mvsa_score3 = []
for i in range(len(pretrained_names)):
    print('Both MVSA:', pretrained_names[i])
    history, scores = run_and_evaluate('single-' + pretrained_names[i], mvsa_features[i], mvsa_image_labels, verbose=0)
    mvsa_hitories3.append(history)
    mvsa_score3.append(scores)
    print()
    
df_scores3 = pd.DataFrame(mvsa_score3, columns=['Loss', 'Accuracy', 'F1-score'], index=pretrained_names)

With original image labels
Both MVSA: vgg16
26

Both MVSA: vgg19
11

Both MVSA: resnet50
14

Both MVSA: resnet101
14

Both MVSA: resnet152
14

Both MVSA: densenet121
14

Both MVSA: densenet169
14

Both MVSA: densenet201
14



In [26]:
print('With multimodal labels')
mvsa_hitories4 = []
mvsa_score4 = []
for i in range(len(pretrained_names)):
    print('Both MVSA:', pretrained_names[i])
    history, scores = run_and_evaluate('single-' + pretrained_names[i], mvsa_features[i], mvsa_multimodal_labels, verbose=0)
    mvsa_hitories4.append(history)
    mvsa_score4.append(scores)
    print()
    
df_scores4 = pd.DataFrame(mvsa_score4, columns=['Loss', 'Accuracy', 'F1-score'], index=pretrained_names)

With multimodal labels
Both MVSA: vgg16
11

Both MVSA: vgg19
11

Both MVSA: resnet50
14

Both MVSA: resnet101
14

Both MVSA: resnet152
14

Both MVSA: densenet121
14

Both MVSA: densenet169
14

Both MVSA: densenet201
18



In [27]:
print('With both MVSA merged together\n')
display_dataframes((style_dataframe(df_scores3), style_dataframe(df_scores4)), 
                   names=['Original image labels', 'Multimodal labels'])

With both MVSA merged together



Original image labels Multimodal labels 
 
 
 
   
 Loss 
 Accuracy 
 F1-score 
 
 
 
 
 vgg16 
 0.859472 
 0.591787 
 0.591787 
 
 
 vgg19 
 0.873409 
 0.585507 
 0.585507 
 
 
 resnet50 
 0.878242 
 0.576812 
 0.576812 
 
 
 resnet101 
 0.881290 
 0.576812 
 0.576812 
 
 
 resnet152 
 0.881441 
 0.570048 
 0.570048 
 
 
 densenet121 
 0.854309 
 0.593720 
 0.593720 
 
 
 densenet169 
 0.855433 
 0.586957 
 0.586957 
 
 
 densenet201 
 0.852574 
 0.597585 
 0.597585 
 
 
 
 
 
 
 
   
 Loss 
 Accuracy 
 F1-score 
 
 
 
 
 vgg16 
 0.847671 
 0.662319 
 0.662319 
 
 
 vgg19 
 0.851658 
 0.663768 
 0.663768 
 
 
 resnet50 
 0.838595 
 0.661836 
 0.661836 
 
 
 resnet101 
 0.840538 
 0.663285 
 0.663285 
 
 
 resnet152 
 0.842235 
 0.663285 
 0.663285 
 
 
 densenet121 
 0.827224 
 0.651208 
 0.651208 
 
 
 densenet169 
 0.820361 
 0.657488 
 0.657488 
 
 
 densenet201 
 0.841692 
 0.664251 
 0.664251